# Ejemplo: Entrenamiento de Modelos Recurrentes

Este notebook demuestra cómo usar los módulos organizados para entrenar modelos RNN, LSTM y GRU.

## Contenido
1. Importar módulos
2. Preparar datos sintéticos
3. Entrenar modelo LSTM
4. Evaluar y visualizar resultados

In [ ]:
# Imports
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Imports de nuestros módulos
from models import LSTMRegressor, LSTMClassifier
from utils import (
    make_windows_regression,
    TimeSeriesDataset,
    train_model,
    plot_training_history,
    plot_predictions_vs_actual
)

print("Módulos importados correctamente")

In [ ]:
# Configuración
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Parámetros
T = 5000
WINDOW_SIZE = 32
HORIZON = 1
BATCH_SIZE = 64
EPOCHS = 30  # Reducido para ejemplo rápido
LEARNING_RATE = 0.001
RANDOM_SEED = 42

# Establecer semillas
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

## 1. Generar datos sintéticos

In [ ]:
# Generar serie temporal sintética
t = np.arange(T, dtype=np.float32)
y = 0.75 * np.sin(2 * np.pi * t / 60) + 0.01 * np.random.randn(T)

# Visualizar
plt.figure(figsize=(12, 4))
plt.plot(t[:500], y[:500])
plt.title('Serie Temporal Sintética (primeros 500 puntos)')
plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.grid(True, alpha=0.3)
plt.show()

print(f'Generados {T} puntos de la serie temporal')

## 2. Preparar datos

In [ ]:
# Dividir datos
n_train = int(0.7 * T)
n_val = int(0.2 * T)

y_train = y[:n_train]
y_val = y[n_train:n_train + n_val]
y_test = y[n_train + n_val:]

# Normalizar
y_mu = y_train.mean()
y_sd = y_train.std()

y_train_z = (y_train - y_mu) / y_sd
y_val_z = (y_val - y_mu) / y_sd
y_test_z = (y_test - y_mu) / y_sd

print(f'Train: {len(y_train)}, Val: {len(y_val)}, Test: {len(y_test)}')
print(f'Media: {y_mu:.4f}, Desv. estándar: {y_sd:.4f}')

In [ ]:
# Crear ventanas deslizantes
Xtr, Ytr = make_windows_regression(y_train_z, WINDOW_SIZE, HORIZON)
Xvl, Yvl = make_windows_regression(y_val_z, WINDOW_SIZE, HORIZON)
Xts, Yts = make_windows_regression(y_test_z, WINDOW_SIZE, HORIZON)

print(f'Xtr: {Xtr.shape}, Ytr: {Ytr.shape}')
print(f'Xvl: {Xvl.shape}, Yvl: {Yvl.shape}')
print(f'Xts: {Xts.shape}, Yts: {Yts.shape}')

In [ ]:
# Crear DataLoaders
train_loader = DataLoader(
    TimeSeriesDataset(Xtr, Ytr),
    batch_size=BATCH_SIZE,
    shuffle=True
)
val_loader = DataLoader(
    TimeSeriesDataset(Xvl, Yvl),
    batch_size=BATCH_SIZE,
    shuffle=False
)
test_loader = DataLoader(
    TimeSeriesDataset(Xts, Yts),
    batch_size=BATCH_SIZE,
    shuffle=False
)

print(f'DataLoaders creados con batch_size={BATCH_SIZE}')

## 3. Crear y entrenar modelo LSTM

In [ ]:
# Crear modelo
model = LSTMRegressor(
    in_dim=1,
    hidden=128,
    layers=2,
    out_dim=HORIZON,
    bidirectional=False,
    dropout=0.0
).to(device)

print(model)
print(f'\nNúmero de parámetros: {sum(p.numel() for p in model.parameters()):,}')

In [ ]:
# Configurar entrenamiento
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Entrenar
print(f'Entrenando por {EPOCHS} épocas...\n')
history, best_state = train_model(
    model, train_loader, val_loader,
    criterion, optimizer, EPOCHS, device,
    clip_norm=1.0, verbose=True
)

# Cargar mejor modelo
model.load_state_dict(best_state)
print('\n✓ Entrenamiento completado')

## 4. Evaluar y visualizar resultados

In [ ]:
# Visualizar historial de entrenamiento
plot_training_history(history)

In [ ]:
# Hacer predicciones en conjunto de prueba
model.eval()
test_predictions = []
test_targets = []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        pred = model(xb)
        test_predictions.append(pred.cpu().numpy())
        test_targets.append(yb.numpy())

y_pred = np.concatenate(test_predictions, axis=0).flatten()
y_true = np.concatenate(test_targets, axis=0).flatten()

# Calcular métricas
mse = np.mean((y_pred - y_true) ** 2)
rmse = np.sqrt(mse)
r2 = 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - y_true.mean()) ** 2)

print(f'Test MSE: {mse:.6f}')
print(f'Test RMSE: {rmse:.6f}')
print(f'Test R²: {r2:.6f}')

In [ ]:
# Visualizar predicciones vs valores reales
plot_predictions_vs_actual(y_true, y_pred)

In [ ]:
# Visualizar secuencia de predicciones
n_samples = 200
plt.figure(figsize=(14, 5))
plt.plot(range(n_samples), y_true[:n_samples], label='Real', marker='o', markersize=3, alpha=0.7)
plt.plot(range(n_samples), y_pred[:n_samples], label='Predicho', marker='x', markersize=3, alpha=0.7)
plt.xlabel('Muestra')
plt.ylabel('Valor Normalizado')
plt.title(f'Predicción de Series Temporales (primeros {n_samples} puntos)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Guardar modelo
torch.save(best_state, 'lstm_regressor_ejemplo.pth')
print('✓ Modelo guardado como lstm_regressor_ejemplo.pth')

## Conclusiones

Este notebook demostró:
1. ✅ Cómo importar y usar los módulos organizados
2. ✅ Preparación de datos con ventanas deslizantes
3. ✅ Entrenamiento de un modelo LSTM para regresión
4. ✅ Evaluación con métricas (MSE, RMSE, R²)
5. ✅ Visualización de resultados

Para clasificación, consulta `train_classification.py` o adapta este notebook usando `LSTMClassifier` en lugar de `LSTMRegressor`.